# Projeto de Precificação de Imóveis de São Paulo

## Etapa 1: Extração e Tratamento dos dados

Na primeira etapa do projeto, será realizado a extração e tratamento das bases de dados dos imóveis do município de São Paulo de modo a criar uma base de dados consistente para ser utilizada na etapa de análise exploratória, são eles:

* Dados do valor dos imovéis (Fonte: Kaggle)
* Dados dos endereços 
* Dados do censo imobiliário (Fonte: IBGE)
* Dados geográficos (Fonte: IBGE)

### Dados dos imóveis

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "https://raw.githubusercontent.com/rafaelladuarte/precificacao_imoveis/main/data_properties_SP.csv"
df_imovel = pd.read_csv(path)
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price
436,"Rua Januário Miraglia, 148","Vila Nova Conceição, São Paulo",180,3,1,4,R$ 2.980.000
9817,"Vila Nair, São Paulo",SP,210,3,2,2,R$ 940.000
4389,"Jardim Guedala, São Paulo",SP,429,4,6,8,R$ 25.000\n /Mês
5467,Rua Doutor Jacques Tupinambá,"Vila Baby, São Paulo",103,3,2,2,R$ 390.000
3721,Rua Nicolau Zarvos,"Parque Jabaquara, São Paulo",199,4,4,4,R$ 1.350.000


Limpando os espaços vazios extras nos valores dos campos

In [3]:
df_imovel = df_imovel.applymap(
    lambda x: x.strip() if isinstance(x, str) else x
)

Nota-se que o dataframe não possui a coluna que nos diz a respeito em qual bairro da cidade de São Paulo o imóvel localiza. No entanto, é possivel observar que essa informação pode ser encontrada na coluna Street ou City.

Ao separar os valores do campo Street por virgula, nota-se que pode ocorrer 3 partições

In [4]:
df_imovel["Street"].str.split(pat=",",expand = True)

,0,1,2
0,Avenida Itacira,255,None
1,Rua Aurelia Perez Alvarez,42,None
2,Rua Alba Valdez,None,None
3,Jardim Morumbi,São Paulo,None
4,Rua Tobias Barreto,195,None
...,...,...,...
10028,Rua Tapuçu,None,None
10029,Rua Guararema,None,None
10030,Rua Estero Belaco,None,None
10031,Rua Manuel Onha,,514


Inserindo as partições da separação no dataframe

In [5]:
df_imovel[
    [
        "Street_1",
        "Street_2", 
        "Street_3"
    ]
]  = df_imovel["Street"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [6]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3
5817,Rua Irmão Gonçalo,"Jardim das Bandeiras, São Paulo",815,6,6,4,R$ 4.900.000,Rua Irmão Gonçalo,None,None
1086,Rua Antônio de Gouveia Giudice,"Alto de Pinheiros, São Paulo",291,3,4,3,R$ 3.900.000,Rua Antônio de Gouveia Giudice,None,None
9442,Rua Cotoxó,"Perdizes, São Paulo",255,4,4,3,R$ 2.868.500,Rua Cotoxó,None,None
5628,Rua Doutor João Pinheiro,"Jardim Paulista, São Paulo",457,5,6,3,R$ 6.500.000,Rua Doutor João Pinheiro,None,None
9848,"Jardim Europa, São Paulo",SP,870,5,5,3,R$ 12.600.000,Jardim Europa,São Paulo,None


Já ao separar os valores do campo City por virgula, ocorrer apenas 2 partições

In [7]:
df_imovel["City"].str.split(pat=",",expand = True)

,0,1
0,Planalto Paulista,São Paulo
1,Jardim dos Estados,São Paulo
2,Jardim Reimberg,São Paulo
3,SP,None
4,Mooca,São Paulo
...,...,...
10028,Vila Sofia,São Paulo
10029,Bosque da Saúde,São Paulo
10030,Vila da Saúde,São Paulo
10031,Vila Oratório,São Paulo


Inserindo as partições da separação no dataframe

In [8]:
df_imovel[["City_1", "City_2"]]  = df_imovel["City"].str.split(pat=",",expand = True)

Visualizando as partições no dataframe

In [9]:
df_imovel.sample(5)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
9865,Rua Agostinho Gomes,"Ipiranga, São Paulo",150,2,3,2,R$ 930.000,Rua Agostinho Gomes,None,None,Ipiranga,São Paulo
8924,"Planalto Paulista, São Paulo",SP,240,3,2,1,R$ 950.000,Planalto Paulista,São Paulo,None,SP,None
7449,"Jardim da Glória, São Paulo",SP,750,6,9,6,R$ 2.880.000,Jardim da Glória,São Paulo,None,SP,None
7187,"Vila Sônia, São Paulo",SP,121,2,3,2,R$ 750.000,Vila Sônia,São Paulo,None,SP,None
9248,Rua Francisco Preto,"Vila Morse, São Paulo",150,3,3,2,R$ 3.600\n /Mês,Rua Francisco Preto,None,None,Vila Morse,São Paulo


Atualizando o campo City para São Paulo, cidade dos dados que está sendo analisado

In [10]:
df_imovel["City"] = "São Paulo"

In [11]:
df_imovel.sample(2)

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2
5700,Rua Maria Leonete da Silva Nóbrega,São Paulo,270,4,6,5,R$ 1.950.000,Rua Maria Leonete da Silva Nóbrega,None,None,Vila Ida,São Paulo
6816,Avenida Albert Einstein,São Paulo,790,5,9,9,R$ 6.000.000,Avenida Albert Einstein,None,None,Jardim Leonor,São Paulo


Criando a coluna Bairro

In [12]:
df_imovel["Bairro"] = df_imovel["City_1"]

Removendo valores do campo Street, quando a informação representa um bairro

In [13]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Street"
] = np.nan

Inserindo o valor do Bairro na sua respectiva coluna

In [14]:
df_imovel.loc[
    df_imovel["Bairro"] == "SP", "Bairro"
] = df_imovel['Street_1']

In [15]:
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Street_1,Street_2,Street_3,City_1,City_2,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Avenida Itacira,255,None,Planalto Paulista,São Paulo,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Rua Aurelia Perez Alvarez,42,None,Jardim dos Estados,São Paulo,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Rua Alba Valdez,None,None,Jardim Reimberg,São Paulo,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,São Paulo,None,SP,None,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Rua Tobias Barreto,195,None,Mooca,São Paulo,Mooca


Removendo as colunas desnecessarias

In [16]:
df_imovel = df_imovel.drop(
    columns = [
        'Street_1',
        'Street_2',	
        'Street_3',
        'City_1',
        'City_2',
    ]
)
df_imovel.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price,Bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


In [17]:
df_imovel.columns = ['rua', 'cidade','metragem','quartos','banheiros','vagas','preco','bairro']

In [18]:
df_imovel.head()

,rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca


Tratando os valores da coluna preço

Nota-se que elas estão no tipo string

In [19]:
type(df_imovel["preco"][0])

str

Separando a string por espaço

In [20]:
df_imovel["preco"].str.split(expand = True)

,0,1,2
0,R$,7.000.000,None
1,R$,3.700.000,None
2,R$,380.000,None
3,R$,685.000,None
4,R$,540.000,None
...,...,...,...
10028,R$,665.000,None
10029,R$,2.300.000,None
10030,R$,1.050.000,None
10031,R$,2.200,/Mês


In [21]:
df_imovel["preco"].str.split(expand = True)[2].unique()

array([None, '/Mês', '/Ano', '/Dia'], dtype=object)

In [22]:
df_imovel[["moeda", "preco_anuncio", "tipo_anuncio"]] = df_imovel["preco"].str.split(expand = True)

In [23]:
df_imovel.sample(3)

,rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
5538,NaN,São Paulo,900,5,2,8,R$ 25.000\n /Mês,Morumbi,R$,25.000,/Mês
7071,Rua Emílio Pedutti,São Paulo,615,4,6,6,R$ 3.700.000,Morumbi,R$,3.700.000,None
9252,Rua João Roman Wirkus,São Paulo,112,2,2,2,R$ 320.000,Jardim Rio Pequeno,R$,320.000,None


In [24]:
df_imovel[df_imovel["tipo_anuncio"].isnull()]["tipo_anuncio"].unique()

array([None], dtype=object)

In [25]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10033 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rua            6596 non-null   object
 1   cidade         10033 non-null  object
 2   metragem       10033 non-null  object
 3   quartos        10033 non-null  object
 4   banheiros      10033 non-null  object
 5   vagas          10033 non-null  object
 6   preco          10033 non-null  object
 7   bairro         10033 non-null  object
 8   moeda          10033 non-null  object
 9   preco_anuncio  10033 non-null  object
 10  tipo_anuncio   852 non-null    object
dtypes: object(11)
memory usage: 862.3+ KB


In [26]:
df_imovel = df_imovel[df_imovel["tipo_anuncio"].isnull()]
df_imovel

,rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,R$ 7.000.000,Planalto Paulista,R$,7.000.000,None
1,"Rua Aurelia Perez Alvarez, 42",São Paulo,469-524,3-6,3-6,4,R$ 3.700.000,Jardim dos Estados,R$,3.700.000,None
2,Rua Alba Valdez,São Paulo,125,4,3,2,R$ 380.000,Jardim Reimberg,R$,380.000,None
3,NaN,São Paulo,310,3,2,4,R$ 685.000,Jardim Morumbi,R$,685.000,None
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,R$ 540.000,Mooca,R$,540.000,None
...,...,...,...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,R$ 1.500.000,Planalto Paulista,R$,1.500.000,None
10028,Rua Tapuçu,São Paulo,90,2,2,2,R$ 665.000,Vila Sofia,R$,665.000,None
10029,Rua Guararema,São Paulo,600,8,7,5,R$ 2.300.000,Bosque da Saúde,R$,2.300.000,None
10030,Rua Estero Belaco,São Paulo,200,3,3,6,R$ 1.050.000,Vila da Saúde,R$,1.050.000,None


In [27]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rua            6129 non-null   object
 1   cidade         9181 non-null   object
 2   metragem       9181 non-null   object
 3   quartos        9181 non-null   object
 4   banheiros      9181 non-null   object
 5   vagas          9181 non-null   object
 6   preco          9181 non-null   object
 7   bairro         9181 non-null   object
 8   moeda          9181 non-null   object
 9   preco_anuncio  9181 non-null   object
 10  tipo_anuncio   0 non-null      object
dtypes: object(11)
memory usage: 860.7+ KB


Substituindo o ponto dos milhares para vazio, pra conseguir converter os valores para float

In [28]:
df_imovel["preco_anuncio"].str.replace(".","",regex=False).astype(float)

0        7000000.0
1        3700000.0
2         380000.0
3         685000.0
4         540000.0
           ...    
10027    1500000.0
10028     665000.0
10029    2300000.0
10030    1050000.0
10032     270000.0
Name: preco_anuncio, Length: 9181, dtype: float64

Adicionando uma coluna no dataframe para o preço do anuncio em formato float

In [29]:
df_imovel["preco_anuncio_float"] = df_imovel["preco_anuncio"].str.replace(".","",regex=True).astype(float)

In [30]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rua                  6129 non-null   object 
 1   cidade               9181 non-null   object 
 2   metragem             9181 non-null   object 
 3   quartos              9181 non-null   object 
 4   banheiros            9181 non-null   object 
 5   vagas                9181 non-null   object 
 6   preco                9181 non-null   object 
 7   bairro               9181 non-null   object 
 8   moeda                9181 non-null   object 
 9   preco_anuncio        9181 non-null   object 
 10  tipo_anuncio         0 non-null      object 
 11  preco_anuncio_float  9181 non-null   float64
dtypes: float64(1), object(11)
memory usage: 932.4+ KB


In [31]:
df_imovel.sample(5)

,rua,cidade,metragem,quartos,banheiros,vagas,preco,bairro,moeda,preco_anuncio,tipo_anuncio,preco_anuncio_float
9795,NaN,São Paulo,205,3,3,2,R$ 740.000,Ipiranga,R$,740.000,None,740000.0
6986,"Rua Godofredo Braga, 39",São Paulo,126,2,2,3,R$ 379.000,Vila Fachini,R$,379.000,None,379000.0
473,Rua Mauricina,São Paulo,150,2,3,3,R$ 1.280.000,Vila Romana,R$,1.280.000,None,1280000.0
1436,NaN,São Paulo,361,4,4,3,R$ 4.200.000,Jardim dos Estados,R$,4.200.000,None,4200000.0
3235,NaN,São Paulo,266,3,3,3,R$ 2.000.000,Vila Ipojuca,R$,2.000.000,None,2000000.0


Removendo as colunas desnecessarias

In [32]:
df_imovel = df_imovel.drop(
    columns = [
        'tipo_anuncio',
        'moeda',
        'preco_anuncio',
        'preco'
    ]
)

In [33]:
df_imovel['metragem']

0           1000
1        469-524
2            125
3            310
4            100
          ...   
10027        262
10028         90
10029        600
10030        200
10032        117
Name: metragem, Length: 9181, dtype: object

Substituindo os valores que não são numeros nas colunas de metragem e vagas para NaN que significa Not a Number

In [34]:
df_imovel['metragem'] = df_imovel['metragem'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['metragem']

0        1000
1         NaN
2         125
3         310
4         100
         ... 
10027     262
10028      90
10029     600
10030     200
10032     117
Name: metragem, Length: 9181, dtype: object

In [35]:
df_imovel['vagas'] = df_imovel['vagas'].apply(
    lambda x: np.nan if '-' in x else x
)
df_imovel['vagas']

0        6
1        4
2        2
3        4
4        2
        ..
10027    4
10028    2
10029    5
10030    6
10032    2
Name: vagas, Length: 9181, dtype: object

Removendo as linhas que possuem valores nulos nas colunas de metragem e vagas

In [36]:
df_imovel = df_imovel.dropna(subset=["metragem","vagas"])
df_imovel

,rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
0,"Avenida Itacira, 255",São Paulo,1000,4,8,6,Planalto Paulista,7000000.0
2,Rua Alba Valdez,São Paulo,125,4,3,2,Jardim Reimberg,380000.0
3,NaN,São Paulo,310,3,2,4,Jardim Morumbi,685000.0
4,"Rua Tobias Barreto, 195",São Paulo,100,3,2,2,Mooca,540000.0
5,Rua Graham Bell,São Paulo,440,4,4,6,Santo Amaro,1980000.0
...,...,...,...,...,...,...,...,...
10027,Avenida Odila,São Paulo,262,3,6,4,Planalto Paulista,1500000.0
10028,Rua Tapuçu,São Paulo,90,2,2,2,Vila Sofia,665000.0
10029,Rua Guararema,São Paulo,600,8,7,5,Bosque da Saúde,2300000.0
10030,Rua Estero Belaco,São Paulo,200,3,3,6,Vila da Saúde,1050000.0


In [37]:
df_imovel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9177 entries, 0 to 10032
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rua                  6125 non-null   object 
 1   cidade               9177 non-null   object 
 2   metragem             9177 non-null   object 
 3   quartos              9177 non-null   object 
 4   banheiros            9177 non-null   object 
 5   vagas                9177 non-null   object 
 6   bairro               9177 non-null   object 
 7   preco_anuncio_float  9177 non-null   float64
dtypes: float64(1), object(7)
memory usage: 645.3+ KB


Mudando o tipo dos valores das colunas metragem, quartos, banheiros e vagas para inteiro

In [38]:
df_imovel["metragem"] = df_imovel["metragem"].astype(int)
df_imovel["quartos"] = df_imovel["quartos"].astype(int)
df_imovel["banheiros"] = df_imovel["banheiros"].astype(int)
df_imovel["vagas"] = df_imovel["vagas"].astype(int)

In [39]:
df_imovel.sample(5)

,rua,cidade,metragem,quartos,banheiros,vagas,bairro,preco_anuncio_float
1423,"Avenida Alfredo Zunkeller, 156",São Paulo,145,3,3,6,Parque Mandaqui,1300000.0
1110,Rua Doutor Tomás Alves,São Paulo,350,3,3,2,Vila Mariana,2500000.0
5986,Rua Mota Pais,São Paulo,188,3,2,2,Vila Ipojuca,1270000.0
170,NaN,São Paulo,300,3,3,3,Siciliano,1300000.0
7263,NaN,São Paulo,793,4,7,5,Interlagos,2680000.0


Exportando o dataframe dos valores dos imoveis para um arquivo csv

In [40]:
df_imovel.to_csv('dados_imoveis_tratados.csv')